In [ ]:
#!pip install wandb
#import wandb
#wandb.init()

In [ ]:
import os
import cv2
import math
import pafy
import random
import numpy as np
import datetime as dt
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

!pip install -q -U keras-tuner
from kerastuner.tuners import RandomSearch
import keras


In [ ]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
%cd 'My Drive'

%cd 'Action Recognition'


In [ ]:
%ls

In [ ]:
#Constants

datasetName= 'hmdb51'

In [ ]:
image_height, image_width = 64, 64
images_per_class = 8000
dataset_directory = "hmdb51"
classes_list = ["golf", "eat", "dribble", "climb", "clap", "run"]
model_output_size = len(classes_list)


In [ ]:
def frames_extraction(video_path):  #helper function to extract frames from videos
    frames_list = []                #empty list for frames
    video_reader = cv2.VideoCapture(video_path) #Read frames from video
    while True:     #Iterate through frames
        success, frame = video_reader.read() #Whilst frames are available
        if not success:
            break
        resized_frame = cv2.resize(frame, (image_height, image_width))    #Resize to pre-defined pixel values
        normalized_frame = resized_frame / 255                            #Normalize frames to range 0-1
        frames_list.append(normalized_frame)                              #Add to frame list for this video
    video_reader.release()                                                #Close and release contents
    return frames_list    

In [ ]:
def create_dataset():    #Create dataset function
    temp_features = [] #empty list to hold each videos frames
    features = []  #final list of frames will be in this list
    labels = []  #Final list of labels will be in this list

    for class_index, class_name in enumerate(classes_list):  #Iterate through chosen classes
        print(f'Extracting Data of Class: {class_name}')        
        files_list = os.listdir(os.path.join(dataset_directory, class_name)) #Got to class folder and get video list
        for file_name in files_list:
            video_file_path = os.path.join(dataset_directory, class_name, file_name)
            frames = frames_extraction(video_file_path)   #Extract frames for current video
            temp_features.extend(frames)      #Add to temp frames list

        features.extend(random.sample(temp_features, images_per_class))       #Choose 8000 random frames from current class (all videos in class)
        labels.extend([class_index] * images_per_class)                         #Assign 8000 labels correctly
        temp_features.clear()

    features = np.asarray(features)   #Convert both to numpy array
    labels = np.array(labels)  
    return features, labels

In [ ]:
features, labels = create_dataset()

In [ ]:
print (features.shape)
print (labels.shape)
#print (features)

In [ ]:
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)

In [ ]:
def create_model(hp):

    model = Sequential()

    # Defining The Model Architecture
    model.add(Conv2D(filters = hp.Int('Conv2D Units', min_value=32, max_value=256, step=32), kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    
    for i in range(hp.Int('Conv2D Layers', 1, 4)):
      model.add(Conv2D(filters = hp.Int('Conv2D Units', min_value=32, max_value=256, step=32), kernel_size = (3, 3), activation = 'relu'))
    
    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    
    for i in range(hp.Int('Fully Connected Layers', 1, 3)):
      model.add(Dense(units=hp.Int('Dense Units',min_value=32,max_value=512,step=32), activation = 'relu'))
    
    
    model.add(BatchNormalization())


    model.add(Dense(model_output_size, activation = 'softmax'))

    # Printing the models summary
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        )

    return model





In [ ]:
tuner = RandomSearch(
    create_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='project',
    project_name='HAR')

tuner.search_space_summary()

tuner.search(features_train, labels_train,
             epochs=40,
             validation_split=0.2, callbacks=[EarlyStopping(patience=7)])

In [ ]:
tuner.search_space_summary()


In [ ]:
tuner.results_summary()